In [ ]:
import SimpleITK as sitk
#import registration_utilities as ru
from downloaddata import fetch_data as fdata
%matplotlib notebook
import matplotlib.pyplot as plt
import gui
import registration_callbacks as rc
import registration_gui as rgui
import registration_utilities as ru
import utilities 

import numpy as np
import os
OUTPUT_DIR = 'output'

In [ ]:
fixed_image = sitk.ReadImage("Data/Pa1_PET.mha", sitk.sitkFloat64)
fixed_mask = sitk.ReadImage("Data/Pa1_PET_Mask.mha", sitk.sitkUInt8)

moving_image = sitk.ReadImage("Data/Pa1_MR.mha", sitk.sitkFloat64)
moving_mask = sitk.ReadImage("Data/Pa1_MR_Mask.mha", sitk.sitkUInt8)


In [ ]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

In [ ]:

%%timeit -r1 -n1

global tx

registration_method = sitk.ImageRegistrationMethod()

# Similarity metric settings.
#registration_method.SetMetricAsMeanSquares()
#registration_method.SetMetricAsJointHistogramMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricAsMutualInformationEfficientEntropy(numberOfHistogramBins=50)
#registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)

registration_method.SetInterpolator(sitk.sitkLinear)

# Optimizer settings.
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
#registration_method.SetOptimizerAsConjugateGradientLineSearch(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
#registration_method.SetOptimizerAsOnePlusOneEvolutionary(numberOfIterations=1000)
#registration_method.SetOptimizerAsPowell(numberOfIterations=1000)
#registration_method.SetOptimizerAsAmoeba(numberOfIterations=1000, simplexDelta=True)
registration_method.SetOptimizerAsLBFGS2()
registration_method.SetOptimizerScalesFromPhysicalShift()

# Don't optimize in-place, we would possibly like to run this cell multiple times.
registration_method.SetInitialTransform(initial_transform, inPlace=False)

# Connect all of the observers so that we can perform plotting during registration.
registration_method.AddCommand(sitk.sitkStartEvent, rgui.start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, rgui.end_plot)
#registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, rgui.update_multires_iterations) 
registration_method.AddCommand(sitk.sitkIterationEvent, lambda: rgui.plot_values(registration_method))

tx = registration_method.Execute(fixed_image, moving_image)

# Always check the reason optimization terminated.
print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))

In [ ]:
#Read Fixed and Moving coordinates from text files and generate fixed_points and moving_points 

File1=open("Data/Fixed_coordinates_Pa1.txt","r")
f_p1=File1.read()
f_p1=f_p1[0:-1]
File1.close()

File2=open("Data/Moving_coordinates_Pa1.txt","r")
f_p2=File2.read()
f_p2=f_p2[0:-1]
File2.close()

ele1 = f_p1.split(";")
fixed_points = []
for i in range(0,len(ele1)):    
    a = (ele1[i][1:-1].split(","))    
    tup1 = ()
    for j in range(0,len(a)):
        trm1 = float(a[j].strip())
        tup1 = tup1 + (trm1,)       
    
    fixed_points.insert(i,tup1)
    
ele2 = f_p2.split(";")
moving_points = []
for i in range(0,len(ele2)):    
    b = (ele2[i][1:-1].split(","))    
    tup2 = ()
    for j in range(0,len(b)):
        trm2 = float(b[j].strip())
        tup2 = tup2 + (trm2,)        
    
    moving_points.insert(i,tup2) 
    
#TRE COMPUTATION    
initial_TRE = utilities.target_registration_errors(sitk.Transform(), fixed_points, moving_points)
final_TRE = utilities.target_registration_errors(tx, fixed_points, moving_points)


print('Initial alignment errors in millimeters, mean(std): {:.2f}({:.2f}), max: {:.2f}, median: {:.2f}'.format(np.mean(initial_TRE), 
                                                                                               np.std(initial_TRE), 
                                                                                               np.max(initial_TRE),
                                                                                               np.median(initial_TRE)))
print('Final alignment errors in millimeters, mean(std): {:.2f}({:.2f}), max: {:.2f}, median: {:.2f}'.format(np.mean(final_TRE), 
                                                                                               np.std(final_TRE), 
                                                                                               np.max(final_TRE),
                                                                                               np.median(final_TRE)))  

In [ ]:
# Compute Dice and Hausdorff scores
# Transfer the segmentation via the estimated transformation. Use Nearest Neighbor interpolation to retain the labels.
moving_mask_pre_registration = sitk.Resample(
    moving_mask,
    fixed_mask,
    sitk.Transform(),
    sitk.sitkNearestNeighbor
)

moving_mask_post_registration = sitk.Resample(
    moving_mask,
    fixed_mask,
    tx,
    sitk.sitkNearestNeighbor
)

segmentation_label=100


label_overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
label_overlap_measures_filter.Execute(fixed_mask, moving_mask_pre_registration)
print(
    f"Dice coefficient before registration: {label_overlap_measures_filter.GetDiceCoefficient(segmentation_label):.2f}"
)

label_overlap_measures_filter.Execute(fixed_mask, moving_mask_post_registration)
print(
    f"Dice coefficient after registration: {label_overlap_measures_filter.GetDiceCoefficient(segmentation_label):.2f}"
)

label_overlap_measures_filter.Execute(fixed_mask, moving_mask_pre_registration)
print(
    f"Jaccard coefficient before registration: {label_overlap_measures_filter.GetJaccardCoefficient(segmentation_label):.2f}"
)

label_overlap_measures_filter.Execute(fixed_mask, moving_mask_post_registration)
print(
    f"Jaccard coefficient after registration: {label_overlap_measures_filter.GetJaccardCoefficient(segmentation_label):.2f}"
)

hausdorff_distance_image_filter = sitk.HausdorffDistanceImageFilter()
hausdorff_distance_image_filter.Execute(fixed_mask, moving_mask_pre_registration)
print(
    f"Hausdorff distance before registration: {hausdorff_distance_image_filter.GetHausdorffDistance():.2f}"
)
hausdorff_distance_image_filter.Execute(fixed_mask, moving_mask_post_registration)
print(
    f"Hausdorff distance after registration: {hausdorff_distance_image_filter.GetHausdorffDistance():.2f}"
)